In [1]:
import pandas as pd
import random
import matplotlib.pyplot as plt
import numpy as np

## 读取已去颅骨及预处理的数据集

In [2]:
#reading in a dataframe that contains image arrays, patient IDs ("subject"), and diagnosis
m2 = pd.read_pickle("../../data/ANM/mri_meta_2D_3class.pkl")

In [3]:
m2.head()

,img_array,label,ID_Visit
0,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",2,PRGADC018_m00
1,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",1,TLSMCI612_m00
2,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",2,PRGADC026_m00
3,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",2,THSADC018_m00
4,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",2,TLSADC502_m00


In [4]:
len(m2)

814

In [5]:
m2[m2['ID_Visit'].duplicated(keep=False)].sort_values(by='ID_Visit')

,img_array,label,ID_Visit
350,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",2,KPOADC014_m00
550,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",2,KPOADC014_m00
764,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",2,KPOADC014_m00
652,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",2,KPOADC016_m12
811,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",2,KPOADC016_m12
...,...,...,...
4,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",2,TLSADC502_m00
748,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",2,TLSADC502_m03
567,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",2,TLSADC502_m03
540,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",2,TLSADC510_m00


In [6]:
m2=m2.drop_duplicates(subset='ID_Visit', keep='first')

In [7]:
m2 = m2.reset_index(drop=True)

In [8]:
m2

,img_array,label,ID_Visit
0,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",2,PRGADC018_m00
1,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",1,TLSMCI612_m00
2,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",2,PRGADC026_m00
3,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",2,THSADC018_m00
4,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",2,TLSADC502_m00
...,...,...,...
754,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",2,PRGMCI038_m12
755,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",0,LNDCTL067_m00
756,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",1,PRGMCI004_m00
757,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",1,KPOMCI018_m00


## 观察数据大小

In [9]:
m2['img_array'][0].shape

#三个维度的切片

(3, 72, 72)

## 查看类别分布情况

In [10]:
m2['label'].value_counts()

label
2    323
1    218
0    218
Name: count, dtype: int64

In [11]:
#there are 238 unique patients
subjects = list(set(m2["ID_Visit"].values))
len(subjects)

759

## 独立划分验证集测试集

In [12]:
val = pd.read_pickle("../../processed_data/ANM/overlap/X_val_img.pkl")
test = pd.read_pickle("../../processed_data/ANM/overlap/X_test_img.pkl")

In [13]:
X_val_list=val.index.values.tolist()
X_test_list=test.index.values.tolist()

In [14]:
X_val_list

['LDZCTL008_m00',
 'KPOCTL011_m00',
 'KPOMCI029_m00',
 'LDZCTL017_m12',
 'LNDCTL004_m00',
 'PRGMCI043_m12',
 'LDZADC015_m12',
 'PRGMCI041_m12',
 'KPOADC011_m00',
 'LDZCTL009_m12',
 'LDZADC014_m00',
 'PRGMCI008_m12',
 'KPOMCI034_m12',
 'KPOCTL014_m00',
 'PRGADC006_m00',
 'PRGADC009_m12',
 'PRGADC036_m12',
 'THSMCI037_m00',
 'KPOMCI024_m00',
 'LDZMCI004_m00',
 'KPOADC020_m00',
 'KPOMCI006_m12',
 'PRGCTL008_m12',
 'KPOCTL023_m12',
 'PRGADC003_m00',
 'KPOADC040_m12',
 'PRGADC019_m00',
 'THSADC045_m00',
 'LNDCTL067_m00',
 'KPOMCI001_m12',
 'KPOADC035_m00',
 'LDZCTL004_m12',
 'KPOCTL008_m12',
 'LNDADC015_m12',
 'LNDMCI020_m00',
 'THSADC049_m00',
 'KPOMCI001_m00',
 'THSADC007_m12',
 'THSCTL016_m12',
 'KPOADC008_m00',
 'LDZADC015_m00',
 'LDZCTL009_m00',
 'LDZMCI010_m12',
 'PRGADC036_m00',
 'KPOMCI008_m00',
 'LNDMCI030_m12',
 'KPOCTL008_m00',
 'KPOADC032_m00',
 'PRGCTL032_m12',
 'KPOCTL001_m12',
 'KPOMCI034_m00',
 'LNDCTL061_m12',
 'PRGCTL034_m00',
 'LDZCTL017_m00',
 'PRGMCI018_m12',
 'LDZCTL01

In [15]:
newm2=m2[~m2['ID_Visit'].isin(X_test_list) & ~m2['ID_Visit'].isin(X_val_list)]

In [16]:
X_train=newm2['img_array']

In [17]:
X_train.index=newm2['ID_Visit']

In [18]:
X_train

ID_Visit
PRGADC018_m00    [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...
TLSMCI612_m00    [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...
PRGADC026_m00    [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...
THSADC018_m00    [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...
TLSADC502_m00    [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...
                                       ...                        
THSADC011_m03    [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...
PRGMCI038_m12    [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...
PRGMCI004_m00    [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...
KPOMCI018_m00    [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...
THSCTL023_m12    [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...
Name: img_array, Length: 631, dtype: object

In [19]:
y_train=newm2['label']
y_train.index=newm2['ID_Visit']

In [20]:
y_train

ID_Visit
PRGADC018_m00    2
TLSMCI612_m00    1
PRGADC026_m00    2
THSADC018_m00    2
TLSADC502_m00    2
                ..
THSADC011_m03    2
PRGMCI038_m12    2
PRGMCI004_m00    1
KPOMCI018_m00    1
THSCTL023_m12    0
Name: label, Length: 631, dtype: object

In [21]:
pd.DataFrame(X_train).to_pickle("../../processed_data/ANM/MRI/X_train_img.pkl")
y_train.to_csv('../../processed_data/ANM/MRI/y_train.csv')

## 不独立

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_temp, y_train, y_temp = train_test_split(m2['img_array'], m2['label'], test_size=0.3, stratify=m2['label'])
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp)

In [9]:
pd.DataFrame(X_train).to_pickle("../processed_data/ANM/X_train_img.pkl")
pd.DataFrame(X_val).to_pickle("../processed_data/ANM/X_val_img.pkl")
pd.DataFrame(X_test).to_pickle("../processed_data/ANM/X_test_img.pkl")

In [10]:
y_train.to_csv('../processed_data/ANM/y_train.csv')
y_val.to_csv('../processed_data/ANM/y_val.csv')
y_test.to_csv('../processed_data/ANM/y_test.csv')